<a href="https://colab.research.google.com/github/wcj365/python-stats-dataviz/blob/master/fall2024/10-30-session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10/30/2024

In [1]:
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_rows", None)

## Define reusable functions

In [2]:
FIRST = "First Quartile (<=25%)"
SECOND = "Second Quartile (25-50%)"
THIRD = "Third Quartile (50-75%)"
FOURTH = "Fourth Quartile (>75%)"

def assign_quartile(value, x, y, z):
    if value <= x:
        quartile = FIRST
    elif value > x and value <= y:
        quartile = SECOND
    elif value > y and value <= z:
        quartile = THIRD
    else:
        quartile = FOURTH
    return quartile

## Load and Explore Data

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/wcj365/python-stats-dataviz/refs/heads/master/fall2024/data/World_Development_Indicators_(WDI).csv")

print(df.shape)
df.sample(3)

(4123, 9)


,Year,Country,GDP per capita (current US$),"Life expectancy at birth, total (years)","Population, total",Country Code,Region,Income Group,Lending Type
3678,2015,Timor-Leste,1322.929094,67.137,1205813.0,TLS,East Asia & Pacific,Lower middle income,Blend
642,2019,Cambodia,1671.385400,70.692,16207746.0,KHM,East Asia & Pacific,Lower middle income,IDA
2243,2005,Madagascar,311.793127,60.570,18792171.0,MDG,Sub-Saharan Africa,Low income,IDA


In [4]:
column_names_dict = {
    "GDP per capita (current US$)" : "GDP per Capita",
    "Life expectancy at birth, total (years)" : "Life Expectancy",
    "Population, total" : "Population"
}

df = df.rename(columns=column_names_dict)
df.sample(3)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
768,2012,Chile,15397.780460,79.023000,17341771.0,CHL,Latin America & Caribbean,High income,IBRD
2423,2014,Mexico,11490.021690,74.797000,118755887.0,MEX,Latin America & Caribbean,Upper middle income,IBRD
2686,2011,New Zealand,38387.627078,81.156098,4384000.0,NZL,East Asia & Pacific,High income,Not classified


In [5]:
df.sample(3).T

,2119,1084,251
Year,2014,2005,2008
Country,Lesotho,Ecuador,"Bahamas, The"
GDP per Capita,1165.05065,3014.310009,29013.630287
Life Expectancy,50.033,74.514,72.667
Population,2095242.0,13770012.0,362795.0
Country Code,LSO,ECU,BHS
Region,Sub-Saharan Africa,Latin America & Caribbean,Latin America & Caribbean
Income Group,Lower middle income,Upper middle income,High income
Lending Type,IDA,IBRD,Not classified


## Homework Question 3 (bonus, will cover next week)

step 1 - create a column "Life Expectancy Quartile" similar to the GDP per Capita Quartile above.
Step 2 - Create a column "health and wealth status" based on the following definition
- rich and healthy (1st GDP per capita quartile and 1st life expentancy quartile)
- poor and unhealthy ((4th GDP per capita quartile and 4th life expentancy quartile)
- rich and unhealthy (1st GDP per capita quartile and 4th life expentancy quartile)
- poor and healthy ((4th GDP per capita quartile and 1st life expentancy quartile)
- Other

In [6]:
print(df.shape)
df.sample(5)

(4123, 9)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
1303,2015,France,36652.922305,82.321951,66548272.0,FRA,Europe & Central Asia,High income,Not classified
2857,2011,Palau,10795.564693,NaN,18240.0,PLW,East Asia & Pacific,Upper middle income,IBRD
1037,2015,Djibouti,2409.311902,61.957000,1006259.0,DJI,Middle East & North Africa,Lower middle income,IDA
2668,2012,New Caledonia,37294.022847,77.078049,259000.0,NCL,East Asia & Pacific,High income,Not classified
2939,2017,Peru,6676.308793,75.879000,31605486.0,PER,Latin America & Caribbean,Upper middle income,IBRD


In [7]:
df_2020 = df[df["Year"] == 2020]
print(df_2020.shape)
df_2020.sample(5)

(217, 9)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type
833,2020,Comoros,1519.586780,64.16600,806166.0,COM,Sub-Saharan Africa,Lower middle income,IDA
1612,2020,Haiti,1283.141228,64.05200,11306801.0,HTI,Latin America & Caribbean,Lower middle income,IDA
1517,2020,Guam,34780.861662,76.61200,169231.0,GUM,East Asia & Pacific,High income,Not classified
947,2020,Cuba,9499.572504,77.56700,11300698.0,CUB,Latin America & Caribbean,Upper middle income,Not classified
1308,2020,France,39179.744260,82.17561,67571107.0,FRA,Europe & Central Asia,High income,Not classified


In [8]:
df_2020 = df_2020.dropna(subset=["GDP per Capita", "Life Expectancy"])
df_2020.shape

(202, 9)

In [9]:
# Step 1: Creates 'Life Expectancy Quartile' column and lists the quartile the row falls under.
df_2020['Life Expectancy'].describe()



,Life Expectancy
count,202.000000
mean,72.314599
std,7.451962
min,52.777000
25%,66.779750
50%,72.871500
75%,77.981750
max,85.497561


In [10]:
x = df_2020['Life Expectancy'].describe()['25%']
y = df_2020['Life Expectancy'].describe()['50%']
z = df_2020['Life Expectancy'].describe()['75%']

In [11]:
df_2020['Life Expectancy Quartile'] = df_2020['Life Expectancy'].apply(assign_quartile, args=(x, y, z,))
df_2020.sample(5)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile
2828,2020,Oman,16707.623006,74.757,4.543399e+06,OMN,Middle East & North Africa,High income,Not classified,Third Quartile (50-75%)
1707,2020,India,1913.219733,70.150,1.396387e+09,IND,South Asia,Lower middle income,IBRD,Second Quartile (25-50%)
3987,2020,Vanuatu,2917.756849,70.299,3.116850e+05,VUT,East Asia & Pacific,Lower middle income,IDA,Second Quartile (25-50%)
1555,2020,Guinea,1073.659339,59.327,1.320515e+07,GIN,Sub-Saharan Africa,Lower middle income,IDA,First Quartile (<=25%)
985,2020,Cyprus,28281.425781,81.391,1.237537e+06,CYP,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%)


In [12]:
x = df_2020['GDP per Capita'].describe()['25%']
y = df_2020['GDP per Capita'].describe()['50%']
z = df_2020['GDP per Capita'].describe()['75%']

print(x, y, z)

2188.047693164725 5920.260418960404 20761.21119943435


In [13]:
# Step 2: Creates 'GDP Quartile' column and lists the quartile the row falls under.
# Next, creates 'Health and Wealth Status' column which categorizes a row if
# a country and a year falls under (4) different categories.


df_2020['GDP Quartile'] =  df_2020['GDP per Capita'].apply(assign_quartile, args=(x, y, z,))
df_2020.sample(10)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile
1726,2020,Indonesia,3895.618152,68.808000,271857970.0,IDN,East Asia & Pacific,Upper middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%)
130,2020,Antigua and Barbuda,15224.858589,78.841000,92664.0,ATG,Latin America & Caribbean,High income,IBRD,Fourth Quartile (>75%),Third Quartile (50-75%)
1859,2020,Jamaica,4897.264750,71.869000,2820436.0,JAM,Latin America & Caribbean,Upper middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%)
3322,2020,Slovenia,25558.429054,80.531707,2102419.0,SVN,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%)
1156,2020,Equatorial Guinea,6198.942524,60.708000,1596049.0,GNQ,Sub-Saharan Africa,Upper middle income,IBRD,First Quartile (<=25%),Third Quartile (50-75%)
3797,2020,Turkmenistan,7330.366288,68.687000,6250438.0,TKM,Europe & Central Asia,Upper middle income,IBRD,Second Quartile (25-50%),Third Quartile (50-75%)
3759,2020,Tunisia,3497.719027,75.292000,12161723.0,TUN,Middle East & North Africa,Lower middle income,IBRD,Third Quartile (50-75%),Second Quartile (25-50%)
738,2020,Chad,643.772216,52.777000,16644701.0,TCD,Sub-Saharan Africa,Low income,IDA,First Quartile (<=25%),First Quartile (<=25%)
358,2020,Belgium,45609.003494,80.695122,11538604.0,BEL,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%)
3227,2020,Seychelles,12020.021064,77.236585,98462.0,SYC,Sub-Saharan Africa,High income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%)


In [14]:
def find_status(row):
  if (row['Life Expectancy Quartile'] == FIRST) and (row['GDP Quartile'] == FIRST):
    return 'Poor and unhealthy'
  elif (row['Life Expectancy Quartile'] == FOURTH) and (row['GDP Quartile'] == FIRST):
    return 'Poor but healthy'
  elif (row['Life Expectancy Quartile'] == FOURTH) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and healthy'
  elif (row['Life Expectancy Quartile'] == THIRD) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and somewhat healthy'
  elif (row['Life Expectancy Quartile'] == SECOND) and (row['GDP Quartile'] == FOURTH):
    return 'Rich and somewhat unhealthy'
  elif (row['Life Expectancy Quartile'] == FIRST) and (row['GDP Quartile'] == FOURTH):
    return 'Rich but unhealthy'
  else:
    return 'Other'

df_2020['Health and Wealth Status'] = df_2020.apply(find_status,axis=1)
df_2020.sample(10)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status
339,2020,Belarus,6542.864540,72.457220,9379952.0,BLR,Europe & Central Asia,Upper middle income,IBRD,Second Quartile (25-50%),Third Quartile (50-75%),Other
377,2020,Belize,5185.158070,72.854000,394921.0,BLZ,Latin America & Caribbean,Upper middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%),Other
1232,2020,Ethiopia,918.652594,65.371000,117190911.0,ETH,Sub-Saharan Africa,Low income,IDA,First Quartile (<=25%),First Quartile (<=25%),Poor and unhealthy
301,2020,Bangladesh,2233.305901,71.968000,167420951.0,BGD,South Asia,Lower middle income,IDA,Second Quartile (25-50%),Second Quartile (25-50%),Other
263,2020,"Bahamas, The",23998.268019,72.677000,406471.0,BHS,Latin America & Caribbean,High income,Not classified,Second Quartile (25-50%),Fourth Quartile (>75%),Rich and somewhat unhealthy
1004,2020,Czechia,22992.879383,78.226829,10697858.0,CZE,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
3987,2020,Vanuatu,2917.756849,70.299000,311685.0,VUT,East Asia & Pacific,Lower middle income,IDA,Second Quartile (25-50%),Second Quartile (25-50%),Other
2638,2020,Nepal,1139.189892,69.246000,29348627.0,NPL,South Asia,Lower middle income,IDA,Second Quartile (25-50%),First Quartile (<=25%),Other
2657,2020,Netherlands,52162.570115,81.358537,17441500.0,NLD,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2448,2020,"Micronesia, Fed. Sts.",3639.412699,70.674000,112106.0,FSM,East Asia & Pacific,Lower middle income,IDA,Second Quartile (25-50%),Second Quartile (25-50%),Other


In [15]:
df_2020["Health and Wealth Status"].value_counts()

,count
Health and Wealth Status,
Other,113
Rich and healthy,42
Poor and unhealthy,38
Rich and somewhat healthy,7
Rich and somewhat unhealthy,2


# Countries that belong to each status category.

In [16]:
df_rich_healthy = df_2020[df_2020["Health and Wealth Status"] == "Rich and healthy"]
print(df_rich_healthy.shape)
df_rich_healthy.sample(5)

(42, 12)


,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status
4044,2020,Virgin Islands (U.S.),39411.045254,79.819512,106290.0,VIR,Latin America & Caribbean,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2353,2020,Malta,29597.636163,82.348780,515332.0,MLT,Middle East & North Africa,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2695,2020,New Zealand,41760.594784,82.256098,5090200.0,NZL,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
2999,2020,Portugal,22242.406418,80.975610,10297081.0,PRT,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
1023,2020,Denmark,60836.592412,81.602439,5831404.0,DNK,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy


In [17]:
df_rich_healthy["Country"].unique()

array(['Australia', 'Austria', 'Bahrain', 'Belgium', 'Bermuda', 'Canada',
       'Channel Islands', 'Cyprus', 'Czechia', 'Denmark', 'Estonia',
       'Faroe Islands', 'Finland', 'France', 'Germany',
       'Hong Kong SAR, China', 'Iceland', 'Ireland', 'Isle of Man',
       'Israel', 'Italy', 'Japan', 'Korea, Rep.', 'Liechtenstein',
       'Luxembourg', 'Macao SAR, China', 'Malta', 'Netherlands',
       'New Caledonia', 'New Zealand', 'Norway', 'Portugal',
       'Puerto Rico', 'Qatar', 'Singapore', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'United Arab Emirates', 'United Kingdom',
       'Virgin Islands (U.S.)'], dtype=object)

In [18]:
df_poor_unhealthy = df_2020[df_2020["Health and Wealth Status"] == "Poor and unhealthy"]
print(df_poor_unhealthy.shape)
df_poor_unhealthy["Country"].unique()

(38, 12)


array(['Afghanistan', 'Angola', 'Benin', 'Burkina Faso', 'Burundi',
       'Cameroon', 'Central African Republic', 'Chad', 'Comoros',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Ethiopia', 'Gambia, The',
       'Ghana', 'Guinea', 'Guinea-Bissau', 'Haiti', 'Kenya', 'Lesotho',
       'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritania',
       'Mozambique', 'Niger', 'Nigeria', 'Pakistan', 'Rwanda',
       'Sierra Leone', 'Somalia', 'Sudan', 'Tanzania', 'Togo', 'Uganda',
       'Yemen, Rep.', 'Zambia', 'Zimbabwe'], dtype=object)

In [19]:
df_poor_healthy = df_2020[df_2020["Health and Wealth Status"] == "Poor but healthy"]
print(df_poor_healthy.shape)
df_poor_healthy["Country"].unique()

(0, 12)


array([], dtype=object)

In [20]:
df_rich_unhealthy = df_2020[df_2020["Health and Wealth Status"] == "Rich but unhealthy"]
print(df_rich_unhealthy.shape)
df_rich_unhealthy["Country"].unique()

(0, 12)


array([], dtype=object)

In [21]:
df_rich_unhealthy

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status


In [22]:
df_other = df_2020[df_2020["Health and Wealth Status"] == "Other"]
df_other["Country"].nunique()

113

# Choropleth - Chorom (colorful), pleth -map, place.

Plot population of countries over a map using plotly

In [23]:
# Create the plot
fig = px.choropleth(
    df_2020,
    locations="Country",
    locationmode='country names',
    color="Health and Wealth Status",
    hover_name="Country",
#    color_continuous_scale=px.colors.sequential.Inferno
)

# Show the plot
fig.show()

## Home work Question 1:

Save the Plotly interactive chart to a HTML file.

Make sure specify the option to reduce the size of the file.

In [24]:
fig.write_html("plotly_chart.html", include_plotlyjs='cdn', full_html=False)

# Home work question 2:

Display country names on the Choropleth. TO make the map not messy, try on a dataframe that has only limited number of countries. For example, filter your dataset by only G7 and BRICS countries.

In [25]:
g7_countries = ["Canada", "France", "Germany", "Italy", "Japan", "United Kingdom", "United States"]
brics_countries = ["Brazil", "Russia", "India", "China", "South Africa"]
G7_BRICS_Countires = ['Canada', 'France', 'Germany', 'Italy', 'Japan', 'United Kingdom', 'United States', 'Brazil', 'China', 'India', 'Indonesia', 'Russia', 'South Africa', 'South Korea']

In [26]:
filtered_df = df_2020[df_2020['Country'].isin(G7_BRICS_Countires)]
filtered_df.sample(5)

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status
1308,2020,France,39179.744260,82.175610,6.757111e+07,FRA,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
681,2020,Canada,43349.677856,81.670488,3.800717e+07,CAN,North America,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy
1707,2020,India,1913.219733,70.150000,1.396387e+09,IND,South Asia,Lower middle income,IBRD,Second Quartile (25-50%),First Quartile (<=25%),Other
1726,2020,Indonesia,3895.618152,68.808000,2.718580e+08,IDN,East Asia & Pacific,Upper middle income,IBRD,Second Quartile (25-50%),Second Quartile (25-50%),Other
3379,2020,South Africa,5753.066494,65.252000,5.880193e+07,ZAF,Sub-Saharan Africa,Upper middle income,IBRD,First Quartile (<=25%),Second Quartile (25-50%),Other


In [29]:
import plotly.graph_objects as go
fig = px.choropleth(
    filtered_df,
    locations="Country",
    locationmode='country names',
    color="Country",
    hover_name="Country",
    hover_data=["GDP per Capita", "Life Expectancy"],
    labels={'Country': 'Country'},
    title='G7 and BRICS Countries'
)

fig.add_trace(go.Scattergeo(
    locations=G7_BRICS_Countires,
    locationmode='country names',
    text=G7_BRICS_Countires,
    mode='text',
    name='Country Name'
))


fig.show()

In [30]:
g7_df = df_2020[df_2020['Country'].isin(g7_countries)]
g7_df['Conference'] = 'G7'
g7_df.head()

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status,Conference
681,2020,Canada,43349.677856,81.670488,38007166.0,CAN,North America,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1308,2020,France,39179.744260,82.175610,67571107.0,FRA,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1403,2020,Germany,46749.476228,81.041463,83160871.0,DEU,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1840,2020,Italy,31922.919163,82.195122,59438851.0,ITA,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1878,2020,Japan,39986.928629,84.560000,126261000.0,JPN,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7


In [31]:
brics_df = df_2020[df_2020['Country'].isin(brics_countries)]
brics_df['Conference'] = 'BRICS'
brics_df.head()

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status,Conference
510,2020,Brazil,6923.699912,74.009,2.131963e+08,BRA,Latin America & Caribbean,Upper middle income,IBRD,Third Quartile (50-75%),Third Quartile (50-75%),Other,BRICS
795,2020,China,10408.719554,78.077,1.411100e+09,CHN,East Asia & Pacific,Upper middle income,IBRD,Fourth Quartile (>75%),Third Quartile (50-75%),Other,BRICS
1707,2020,India,1913.219733,70.150,1.396387e+09,IND,South Asia,Lower middle income,IBRD,Second Quartile (25-50%),First Quartile (<=25%),Other,BRICS
3379,2020,South Africa,5753.066494,65.252,5.880193e+07,ZAF,Sub-Saharan Africa,Upper middle income,IBRD,First Quartile (<=25%),Second Quartile (25-50%),Other,BRICS


In [32]:
merged_df = pd.concat([g7_df, brics_df])
merged_df.head()

,Year,Country,GDP per Capita,Life Expectancy,Population,Country Code,Region,Income Group,Lending Type,Life Expectancy Quartile,GDP Quartile,Health and Wealth Status,Conference
681,2020,Canada,43349.677856,81.670488,38007166.0,CAN,North America,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1308,2020,France,39179.744260,82.175610,67571107.0,FRA,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1403,2020,Germany,46749.476228,81.041463,83160871.0,DEU,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1840,2020,Italy,31922.919163,82.195122,59438851.0,ITA,Europe & Central Asia,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7
1878,2020,Japan,39986.928629,84.560000,126261000.0,JPN,East Asia & Pacific,High income,Not classified,Fourth Quartile (>75%),Fourth Quartile (>75%),Rich and healthy,G7


In [34]:

fig = px.choropleth(
    merged_df,
    locations="Country",
    locationmode='country names',
    color="Conference",
    hover_name="Country",
    hover_data=["Conference", "GDP per Capita", "Life Expectancy"],
    labels={'Country': 'Country'},
    title='G7 and BRICS Countries'

)

# Below add_trace block adds labels to choropleth and restricts labels to G7
# and BRICS countries.
fig.add_trace(go.Scattergeo(
    locations=G7_BRICS_Countires,
    locationmode='country names',
    text=G7_BRICS_Countires,
    mode='text',
    name='Country Name'
))

# Show the plot
fig.show()



## Home work Question 3:

Explore the website - https://g7brics.streamlit.app/

